In [1]:
# Jupyter setup to expand cell display to 100% width on your screen (optional)
# Import relevant modules and setup for calling glmnet
%reset -f
%matplotlib inline

from sqlalchemy import create_engine
import sys
import pandas as pd
import numpy as np
import scipy, importlib, pprint, matplotlib.pyplot as plt, warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.impute import KNNImputer

import statsmodels.api as sm
import statsmodels.genmod as genmod
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import mean_absolute_error

from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.model_selection import cross_val_score
import sklearn.model_selection as ms

import warnings
warnings.filterwarnings('ignore')

## Load TSR and attributes data from DB

In [2]:
localhost = {'user': 'postgres', 'password': 'postgres', 'host': 'localhost', 'port': 5432, 'db': 'fiadb'}
params = 'postgresql://{0}:{1}@{2}:{3}/{4}'
engine = create_engine(params.format(localhost['user'], localhost['password'], localhost['host'], localhost['port'], localhost['db']))
# geom_sql = """select distinct grid_id, grid_geom from fs_fiadb.pergrid"""
pergrid_base = """select distinct * from predictor.pergrid_base_eus"""
pergrid_base_df = pd.read_sql(pergrid_base, engine)

In [3]:
pergrid_base_df.columns

Index(['grid_id', 'aet', 'ai', 'art', 'ewd', 'fa', 'map', 'mat', 'mpdq',
       'mtcq', 'pet', 'psn', 'ra', 'rmap', 'rmat', 'tsn', 'mfdf', 'alt', 'shg',
       'mtwq', 'wkb_geometry', 'tsr', 'wa'],
      dtype='object')

## Process TSR predictor and outcome varaibles

### only use lasso-selected variables 

In [4]:
y = pergrid_base_df['tsr']
grid_id = pergrid_base_df['grid_id']
lasso_var= ['aet', 'ai', 'art', 'ewd', 'fa', 'map', 'mat', 'mpdq',
       'mtcq', 'pet', 'psn', 'ra', 'rmap', 'rmat', 'tsn', 'mfdf', 'alt', 'shg',
       'mtwq', 'wa']
pergrid_base_selected_df = pergrid_base_df[lasso_var]

### fill in no-data grid with value from neighboring grids

In [5]:
imputer = KNNImputer(n_neighbors=5)
pergrid_base_selected_filled = imputer.fit_transform(pergrid_base_selected_df)
pergrid_base_df = pd.DataFrame(pergrid_base_selected_filled)

In [6]:
pergrid_base_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,65535.0,1.07484,0.164286,-524.708793,0.0711,1148.5,0.309987,131.5,21.304167,1918.111111,0.713689,1.0,1.833333,0.241667,3.698817,8.088,0.011696,1.2,28.626191,302.9049
1,13144.0,0.65770,8.822390,-894732.416005,0.1062,1208.0,12.476780,128.0,19.837514,1833.026596,32.850457,1.0,4.333333,0.679166,169.350913,3.520,0.774757,4.4,27.766178,149.0013
2,13867.0,0.66700,7.376786,-656234.650346,0.0549,1245.0,10.635709,130.0,19.966610,1850.611285,27.834838,1.0,7.833333,0.945833,143.805179,3.260,0.628627,4.2,27.982256,164.0862
3,65535.0,0.94772,1.734753,-48762.101679,0.0342,1223.0,2.873174,130.5,20.691381,1898.352941,7.071733,2.0,10.083333,0.637500,36.816597,4.760,0.172147,2.0,28.239400,353.2644
4,65535.0,1.07484,0.631731,-6380.557906,0.0027,1183.6,1.045456,249.4,3.757151,1895.387097,2.569308,2.0,4.000000,0.212500,13.451322,4.760,0.068093,1.2,22.364209,121.9797


In [7]:
from sklearn.preprocessing import MinMaxScaler

ss = MinMaxScaler()
X_std = ss.fit_transform(pergrid_base_df)

Xstd=pd.DataFrame(data=X_std[0:,0:],
                index=pergrid_base_df.index,
                columns=lasso_var)

In [8]:
x_train, x_test, y_train, y_test = train_test_split(Xstd, y, test_size=0.2, random_state=12345)

## Calculate VIF

In [18]:
# var_sel = ["aet", "shg", "rmap", "rmat", "fa", "alt"] # lowest overall VIF scores 
# Xstd = Xstd[var_sel]
# Xstd.head()

In [19]:
# if the VIF is between 5-10, multicolinearity is likely present and you should consider dropping the variable.
pd.Series([variance_inflation_factor(Xstd.values, i) 
               for i in range(Xstd.shape[1])], 
              index=Xstd.columns)

aet        2.099093
ai        48.545420
art     1543.590152
ewd      240.599854
fa         5.583221
map       92.286433
mat      425.471442
mpdq      47.314070
mtcq     655.984657
pet      182.075913
psn       21.420803
ra        39.271648
rmap       5.040411
rmat      28.442990
tsn     1254.755912
mfdf      87.922182
alt       28.171383
shg        4.822418
mtwq     517.123907
wa         2.254314
dtype: float64

## Only select factors with low VIF

In [20]:
# var_sel = ["mpdq", "fa", "tsn", "alt", "ra", "pet"]
# Xstd = Xstd[var_sel]

## Build GLM Model

In [9]:
poisson_model = sm.GLM(y_train, x_train, family=sm.families.Poisson())

In [10]:
poisson_results = poisson_model.fit()

In [11]:
print(poisson_results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                    tsr   No. Observations:                 6098
Model:                            GLM   Df Residuals:                     6078
Model Family:                 Poisson   Df Model:                           19
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -21385.
Date:                Sat, 08 Aug 2020   Deviance:                       12553.
Time:                        08:37:19   Pearson chi2:                 1.16e+04
No. Iterations:                     5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
aet           -0.0328      0.016     -2.021      0.0

In [12]:
# null hypothesis: predictors have no effect; A low p-value (< 0.05) indicates that the null hypothesis can be rejected
print("P values of predictors: ")
print(poisson_results.pvalues.sort_values(ascending=True))

P values of predictors: 
fa       0.000000e+00
art     7.958485e-268
ewd     1.239393e-142
tsn     6.578290e-138
alt      1.031806e-36
psn      3.440150e-34
mat      4.496786e-19
ai       4.752147e-17
map      1.215376e-14
wa       1.551797e-13
rmat     1.256971e-06
rmap     2.235561e-06
shg      3.809509e-04
mpdq     7.378883e-04
mtwq     2.512024e-02
mtcq     4.189041e-02
aet      4.322905e-02
ra       5.115956e-01
pet      6.429225e-01
mfdf     7.519936e-01
dtype: float64


In [13]:
print("Predictors which are not statistically significant:")
print(poisson_results.pvalues[poisson_results.pvalues > 0.05])

Predictors which are not statistically significant:
pet     0.642923
ra      0.511596
mfdf    0.751994
dtype: float64


## Build Cross-validation GLM Model

In [14]:
class SMWrapper(BaseEstimator, RegressorMixin):
    """ A universal sklearn-style wrapper for statsmodels GLM w/ Possion """
    def __init__(self, model_class, fit_intercept=True):
        self.model_class = model_class
        self.fit_intercept = fit_intercept
    def fit(self, X, y):
        if self.fit_intercept:
            X = sm.add_constant(X)
        self.model_ = self.model_class(y, X, sm.families.Poisson())
        self.results_ = self.model_.fit()
    def predict(self, X):
        if self.fit_intercept:
            X = sm.add_constant(X)
        return self.results_.predict(X)

In [15]:
wrapped_possion_glm = SMWrapper(sm.GLM)

In [16]:
wrapped_possion_glm.fit(Xstd,y)

## Model Evaluation

In [17]:
import statistics
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

In [18]:
r2_cross_val = cross_val_score(wrapped_possion_glm, Xstd, y, scoring='r2', cv = ms.StratifiedKFold(shuffle = True))
print("Cross-validated R2: ",)
statistics.mean(r2_cross_val.tolist())

Cross-validated R2: 


0.6420685852384757

In [19]:
mae_cross_val = cross_val_score(wrapped_possion_glm, Xstd, y, scoring='neg_mean_absolute_error', cv = ms.StratifiedKFold(shuffle = True))
print("Cross-validated MAE: ",)
statistics.mean(mae_cross_val.tolist())* -1

Cross-validated MAE: 


5.313459795003992

In [20]:
y_train_pred = wrapped_possion_glm.predict(x_train)
print('train MAE', mean_absolute_error(y_train_pred, y_train))
print('train r2', r2_score(y_train, y_train_pred))

train MAE 5.283613637971665
train r2 0.6449113511045662


In [21]:
y_test_pred = wrapped_possion_glm.predict(x_test)
print('test MAE', mean_absolute_error(y_test_pred, y_test))
print('test r2', r2_score(y_test, y_test_pred))

test MAE 5.363486183821899
test r2 0.6422938589035194


In [23]:
glm_y_test_eus = pd.DataFrame(
    {'tsr': y_test,
     'tsr_predicted': y_test_pred})
glm_y_test_eus.to_sql(name='glm_y_test_eus', con=engine, schema='predictor', if_exists='replace', index=False)

In [24]:
glm_y_test.shape

(1525, 2)

## Model Prediction

In [25]:
Y_pred = wrapped_possion_glm.predict(Xstd)

In [26]:
pergrid_all_predicted = pd.DataFrame(
    {'grid_id': grid_id,
     'tsr': y,
     'tsr_predicted': Y_pred.tolist()})

In [27]:
pergrid_all_predicted.head()

,grid_id,tsr,tsr_predicted
0,195,5.0,3.954440
1,656,1.0,8.827576
2,657,6.0,8.895190
3,659,10.0,5.697991
4,660,3.0,5.126189


In [28]:
pergrid_all_predicted.to_sql(name='lasso_glm_eus', con=engine, schema='predictor', if_exists='replace', index=False)

In [29]:
update_geom = """
alter table predictor.lasso_glm_eus add column if not exists wkb_geometry geometry(Polygon,4269);
update predictor.lasso_glm_eus A SET wkb_geometry = B.wkb_geometry
FROM predictor.pergrid_base B
WHERE A.grid_id = B.grid_id
"""

In [30]:
connection = engine.connect()
connection.execute(update_geom)

In [31]:
update_residual = """
alter table predictor.lasso_glm_eus add column residual double precision;
update predictor.lasso_glm_eus set residual = (tsr_predicted-tsr);
"""

In [32]:
connection = engine.connect()
connection.execute(update_residual)